# Итоговый проект. Улицы. Объекты культурного наследия.

In [9]:
# TODO собрать установку всех необходимых модулей в одном месте
!pip install geopandas mapclassify
!pip install osmnx
!pip install keplergl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 37.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922316 sha256=23ddf8b08a68a85937f24b7aeef8c99a24687ef145c903b2358f9fd5403d83a0
  Stored in directory: /root/.cache/pip/wheels/e5/e9/db/b8b8f2fcb5cb1cd7268d88959fce267bc00697cb8f50dcd0d9
Successfully built keplergl


In [67]:
# TODO собрать импорты всех модулей в одном месте
import osmnx as ox
import geopandas as gpd
from keplergl import KeplerGl
from google.colab import output
output.enable_custom_widget_manager()

In [69]:
TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = "Василеостровский район, Санкт-Петербург"  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/DRMPN/python-urban/main/geojson_layers/kgiop_objects.geojson"  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = "https://raw.githubusercontent.com/DRMPN/python-urban/main/geojson_layers/streets.geojson"  # ссылка на слой с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [14]:
# TODO загрузить geodataframe с геометрией для территории TERRITORY_NAME
territory = ox.geocode_to_gdf(TERRITORY_NAME)
# territory.explore()

## Улицы

### Загрузка файла с улицами (Extract)

In [80]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
territory_streets = gpd.read_file(STREETS_FILE_URL, mask=territory)
territory_streets.reset_index(drop=True, inplace=True)

In [ ]:
map = KeplerGl(height=500)
map.add_data(data=territory_streets.copy(), name="Улицы")
map

### Обработка данных с улицами (Transform)

In [82]:
# TODO сгруппировать и объединить геометрии с одинаковыми названиями
territory_streets = territory_streets.dissolve(by="name")

In [ ]:
map = KeplerGl(height=500)
map.add_data(data=territory_streets.copy(), name="Улицы")
map

### Сохранение слоя с улицами (Load)

In [101]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)
df_ts = territory_streets.rename_axis("название")

In [102]:
# TODO сохранить слой в географической проекции в формате GeoJSON
df_ts.to_crs(4326).to_file('territory_streets.geojson', driver='GeoJSON')

## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [94]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
territory_kgiop= gpd.read_file(KGIOP_FILE_URL, mask=territory)

### Обработка объектов культурного наследия (Transform)

In [98]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта
territory_kgiop["lon"] = territory_kgiop.to_crs(3857).geometry.centroid.to_crs(4326).x
territory_kgiop["lat"] = territory_kgiop.to_crs(3857).geometry.centroid.to_crs(4326).y
territory_kgiop.head()

,id,ensemble_name,object_name,occurrence_time,object_location,historical_category,normative_act,object_type,geometry,lon,lat
0,1260,—,Дом А.Р. Гешвенда,1880-1881,"9-я линия В.О., 32",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3370436.885 8386572.161),30.277150,59.940812
1,1021,Дом И.В. Голубина (И.И. Зайцевского),Флигель,нач. 19 в.; 1834,"Репина ул., 17",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3371907.580 8386491.569),30.290361,59.940450
2,1015,—,Дом Г.И. Головкина (А.Н. Чичагова),1720-е - 1730-е; 1878,"1-я линия В.О., 10; Репина ул., 9",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3372020.006 8386392.023),30.291371,59.940002
3,1016,—,Дом Леонтьева М.И. (Шуберта Ф.Ф.),"1722-1741, 1807-1809","1-я линия В.О., 12",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3372009.644 8386417.444),30.291278,59.940116
4,1017,—,Дом А. Сверчкова (Кушелевой),1730-1740-е; 1815; 1863,"1-я линия В.О., 14, лит. А",объект культурного наследия регионального знач...,Распоряжение КГИОП № 139-р от 23.04.2021,Памятник,POINT (3372026.660 8386473.431),30.291431,59.940368


### Сохранение слоя с объектами культурного наследия (Load)

In [106]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
df_tk = territory_kgiop.rename(columns={'id': 'идентификатор',
                                        'ensemble_name': 'имя_ансамбля',
                                        'object_name':'имя_объекта',
                                        'occurrence_time':'время_создания',
                                        'object_location':'расположение_объекта',
                                        'historical_category':'историческая_категория',
                                        'normative_act':'нормативный_акт',
                                        'object_type':'тип_объекта',
                                        'lat':'широта',
                                        'lon':'долгота'})
df_tk.head()

,идентификатор,имя_ансамбля,имя_объекта,время_создания,расположение_объекта,историческая_категория,нормативный_акт,тип_объекта,geometry,долгота,широта
0,1260,—,Дом А.Р. Гешвенда,1880-1881,"9-я линия В.О., 32",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3370436.885 8386572.161),30.277150,59.940812
1,1021,Дом И.В. Голубина (И.И. Зайцевского),Флигель,нач. 19 в.; 1834,"Репина ул., 17",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3371907.580 8386491.569),30.290361,59.940450
2,1015,—,Дом Г.И. Головкина (А.Н. Чичагова),1720-е - 1730-е; 1878,"1-я линия В.О., 10; Репина ул., 9",выявленный объект культурного наследия,Приказ председателя КГИОП № 15 от 20.02.2001,Памятник,POINT (3372020.006 8386392.023),30.291371,59.940002
3,1016,—,Дом Леонтьева М.И. (Шуберта Ф.Ф.),"1722-1741, 1807-1809","1-я линия В.О., 12",объект культурного наследия федерального значения,Постановление Правительства РФ № 527 от 10.07....,Памятник,POINT (3372009.644 8386417.444),30.291278,59.940116
4,1017,—,Дом А. Сверчкова (Кушелевой),1730-1740-е; 1815; 1863,"1-я линия В.О., 14, лит. А",объект культурного наследия регионального знач...,Распоряжение КГИОП № 139-р от 23.04.2021,Памятник,POINT (3372026.660 8386473.431),30.291431,59.940368


In [107]:
# TODO сохранить слой в географической проекции в формате GeoJSON
df_tk.to_crs(4326).to_file('territory_kgiop.geojson', driver='GeoJSON')